In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np

In [2]:
# To display every columns and every row

# pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)

In [3]:
def harvestGlassDoor(pathOfTheChromDriver, requiredJobData,  noOfPages, requiredCountryData = " "):
    # Initializing the chrome browser
    driver = webdriver.Chrome(pathOfTheChromDriver)

    # Opening the URL
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+requiredJobData+"&sc.keyword="+requiredJobData+"&locT=&locId=&jobType="
    driver.get(url)
    
    # Maximizing the opened window
    driver.maximize_window()
    
    # Adding implicit wait
    driver.set_page_load_timeout(30)
    
    # Entering city (or) state (or) country name
    if requiredCountryData == " ":
        driver.find_element_by_id("sc.location").clear()
        driver.find_element_by_xpath("//form[@id='scBar']//button[contains(@class, 'gd-ui-button')]").click()
        
    else:
        driver.find_element_by_id("sc.location").send_keys(requiredCountryData)
        driver.find_element_by_xpath("//form[@id='scBar']//button[contains(@class, 'gd-ui-button')]").click()

    time.sleep(3)

    # To get the Popup
    toGetPopUp = []
    toGetPopUp = driver.find_elements_by_xpath("//*[contains(@class,'react-job-listing')]")    
    toGetPopUp[0].click()

    time.sleep(2)

    # To close the popup
    closeButton = driver.find_element_by_xpath("//*[contains(@class, 'modal_closeIcon-svg')]")
    closeButton.click()

    # Creating a Dataframe to store the data in it.
    data = pd.DataFrame(columns={'Job Title','Salary Estimate', 'Job Description', 'Rating', 'Company Name', 'Location', 'Size', 'Founded', 'Type of Ownership', 'Industry', 'Sector', 'Revenue'})

    # This loop is to get required code from the webpage.

    # Adding implicit wait
    driver.set_page_load_timeout(30)
    
    # To get required number of pages of job data.
    for page in range(noOfPages):
        
        # Time for every page to wait
        driver.set_page_load_timeout(30)

        # Multiple values are taken.
        jobTitles = driver.find_elements_by_xpath("//li[contains(@class, 'react-job-listing')]//a[contains(@class, 'jobLink css-1rd3saf eigr9kq2')]//span")
        companyNames = driver.find_elements_by_xpath("//li[contains(@class, 'react-job-listing')]//a[@class=' css-l2wjgv e1n63ojh0 jobLink']")
        jobLocations = driver.find_elements_by_xpath("//div[contains(@class, 'd-flex flex-wrap css-11d3uq0 e1rrn5ka2')]/span")

        # Locating all the listed jobs
        allJobs = driver.find_elements_by_xpath("//ul[@data-test='jlGrid']//li")

        for i in range(len(allJobs)):
            
            try:
                driver.set_page_load_timeout(30)
            except:
                time.sleep(1) # refresh page manually when any chrome error occurs. (Like Awww Snap!, etc)

            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", allJobs[i])
                driver.execute_script("arguments[0].click();", allJobs[i])
            except:
                time.sleep(1)
            
#             time.sleep(0.5)

            try:
                jobDescriptionText = driver.find_element_by_xpath("//div[contains(@class,'jobDescriptionContent')]").text
            except Exception:
                jobDescriptionText = np.nan

            try:
                jobRatingText = driver.find_element_by_xpath("//span[contains(@class,'css-1m5m32b e1tk4kwz2')]").text
            except Exception:
                jobRatingText = np.nan

            try:
                emplooyeSizeText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[1]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                emplooyeSizeText = np.nan

            try:
                companyFoundedText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[2]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                companyFoundedText = np.nan

            try:
                companyTypeText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[3]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                companyTypeText = np.nan

            try:
                jobIndustryText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[4]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                jobIndustryText = np.nan

            try:
                jobSectorText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[5]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                jobSectorText = np.nan

            try:
                companyRevenueText = driver.find_element_by_xpath("//div[@id='CompanyContainer']//div[@class='d-flex flex-wrap']/div[6]/span[@class='css-i9gxme e1pvx6aw2']").text
            except Exception:
                companyRevenueText = np.nan

            try:
                jobSalaryText = driver.find_element_by_xpath("//span[contains(@class, 'css-56kyx5 css-16kxj2j e1wijj242')]").text
            except Exception:
                jobSalaryText = np.nan

            # Title, company name and job location are directly available so for each job it's corresponding values are assigned
            
            try:
                driver.set_page_load_timeout(30)
                jobTitleText = jobTitles[i].text
            except Exception:
                jobTitleText = np.nan
                    
            
            try:
                driver.set_page_load_timeout(30)
                companyNameText = companyNames[i].text
            except Exception:
                companyNameText = np.nan
                
            try:
                driver.set_page_load_timeout(30)
                jobLocationText = jobLocations[i].text
            except Exception:
                jobLocationText = np.nan

            # Storing all the data into dataset
            data = data.append({'Job Title': jobTitleText, 'Salary Estimate': jobSalaryText, 'Job Description': jobDescriptionText, 'Rating': jobRatingText, 'Company Name': companyNameText, 
                                     'Location': jobLocationText, 'Size': emplooyeSizeText, 'Founded': companyFoundedText, 'Type of Ownership': companyTypeText, 'Industry': jobIndustryText, 
                                     'Sector': jobSectorText, 'Revenue': companyRevenueText},  ignore_index = True)


        driver.set_page_load_timeout(30)
        driver.find_element_by_xpath("//a[@data-test='pagination-next']//span[@class='SVGInline']").click()
        
        time.sleep(3)




    driver.quit()
    
    return data

In [4]:
dataset = harvestGlassDoor(pathOfTheChromDriver="chromedriver.exe", requiredJobData = 'data science', noOfPages = 20, requiredCountryData = 'USA')

In [5]:
dataset.shape

(600, 12)

In [6]:
dataset.to_csv('GlassdoorData_USA10.csv')

In [7]:
dataset.head()

,Founded,Revenue,Location,Job Title,Rating,Company Name,Size,Type of Ownership,Industry,Sector,Salary Estimate,Job Description
0,NaN,NaN,"Chicago, IL",Data Analyst,NaN,Nova Collective,NaN,NaN,NaN,NaN,Employer Provided Salary:$29 - $34 Per Hour,Are you a data analyst who is really excited a...
1,Company - Private,NaN,"Irvine, CA",Data Science Opportunities/ Entry Level for Re...,4.5,PCS Global Tech,501 to 1000 Employees,IT Services,Information Technology,Unknown / Non-Applicable,Employer Provided Salary:$60K,Are you a data analyst who is really excited a...
2,Company - Private,NaN,"Lawrenceville, NJ",Data Science - Entry Level,4.5,CBL Solutions,501 to 1000 Employees,IT Services,Information Technology,Unknown / Non-Applicable,Employer Provided Salary:$60K,"Responsibilities:\n· Provides plan with data, ..."
3,1902,Unknown / Non-Applicable,"Roanoke, VA",Junior Data Scientist,4.4,Titan America,1001 to 5000 Employees,Company - Public,Mining,Mining & Metals,Employer Provided Salary:$66K,"Responsibilities:\n· Provides plan with data, ..."
4,1902,Unknown / Non-Applicable,"New York, NY",110: Head of Data Science,4.4,Dataspace,1001 to 5000 Employees,Company - Public,Mining,Mining & Metals,Employer Provided Salary:$66K,"Titan America, LLC, is a leading environmental..."


## Concatenating all the datasets into single dataset

In [27]:
dataset1 = pd.read_csv('GlassdoorData_USA9.csv')




dataset2 = pd.read_csv('GlassdoorData_USA1.csv')
dataset3 = pd.read_csv('GlassdoorData_USA2.csv')
dataset4 = pd.read_csv('GlassdoorData_USA3.csv')
dataset5 = pd.read_csv('GlassdoorData_USA4.csv')
dataset6 = pd.read_csv('GlassdoorData_USA5.csv')
dataset7 = pd.read_csv('GlassdoorData_USA6.csv')
dataset8 = pd.read_csv('GlassdoorData_USA7.csv')
dataset9 = pd.read_csv('GlassdoorData_USA8.csv')


complete_dataset = pd.concat([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9])


In [4]:
complete_dataset.shape

(6420, 13)

In [5]:
complete_dataset.isnull().sum()

Unnamed: 0              0
Revenue              1858
Company Name           52
Sector               1384
Salary Estimate        76
Location               38
Type of Ownership     925
Industry             1262
Job Title              71
Founded               942
Rating               1456
Job Description       353
Size                 1024
dtype: int64

In [6]:
complete_dataset.fillna('-1')
complete_dataset = complete_dataset[complete_dataset != '-1']

In [9]:
print(complete_dataset.shape)
complete_dataset.head()

(6420, 13)


,Unnamed: 0,Revenue,Company Name,Sector,Salary Estimate,Location,Type of Ownership,Industry,Job Title,Founded,Rating,Job Description,Size
0,0,$10+ billion (USD),Microsoft,Information Technology,NaN,United States,Company - Public,Computer Hardware & Software,Delivery Data Science IC3,1975,4.4,OVERVIEW:\n\nAre you ready to seize and opport...,10000+ Employees
1,1,$10+ billion (USD),Amer Technology In,Information Technology,NaN,"Austin, TX",Company - Public,Computer Hardware & Software,Data Scientist,1975,4.4,OVERVIEW:\n\nAre you ready to seize and opport...,10000+ Employees
2,2,NaN,Newsly Technologies,NaN,Employer Provided Salary:$72 - $77 Per Hour,United States,NaN,NaN,Data Scientist,NaN,NaN,Data Scientist Roles and Responsibilities\n1. ...,NaN
3,3,NaN,Facebook,NaN,Employer Provided Salary:$150K,Remote,NaN,NaN,"Data Scientist, Analytics",NaN,NaN,Company description\nNewsly is an audio news a...,NaN
4,4,$10+ billion (USD),Denken Solutions Inc,Information Technology,NaN,"Irvine, CA",Company - Public,Internet,Data Science (100% Remote),2004,4.3,"As a Data Scientist at Facebook, you will shap...",10000+ Employees


In [10]:
complete_dataset = complete_dataset.drop_duplicates()
complete_dataset.shape

(5479, 13)

In [18]:
complete_dataset.to_csv('GlassdoorData_USA.csv')